In [141]:
import numpy as np
import pandas as pd
import math
import os

In [142]:
MODEL_INIT_FILE = 'model_desc.txt'

<h2>SOFTMAX LAYER</h2>

In [143]:
class Softmax_Layer:
    def __init__(self):
        self.layer_type = 'Softmax'
    
    def __str__(self):
        return f"{self.layer_type} Layer"
    
    def forward(self, X):
        Z = np.exp(X)
        return Z / np.einsum('ij->j', Z)
    
    def backward(self, dZ, learning_rate=0.0001):
        return np.copy(dZ)

<h2>ReLU ACTIVATION </h2>

In [144]:
class ReLU_Activation:
    def __init__(self):
        self.layer_type = 'ReLU'
    
    def __str__(self):
        return f"{self.layer_type} Activation"
    
    def forward(self, X):
        self.X = X

        Z = np.copy(X)
        Z[Z < 0] = 0
        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dX = np.copy(self.X)

        dX[dX < 0] = 0
        dX[dX > 0] = 1
        return dX * dZ

<h2>FULLY CONNECTED LAYER</h2>

In [145]:
class Fully_Connected_Layer:
    def __init__(self, output_dim):
        self.output_dim = output_dim
        self.W = None
        self.b = None

    def __str__(self):
        return f"Fully Connected Layer(output_dim={self.output_dim})"
    
    def forward(self, X):
        self.X = X

        if self.W is None:
            self.W = np.random.randn(X.shape[1], self.output_dim) * math.sqrt(2 / X.shape[0])
        
        if self.b is None:
            self.b = np.zeros((1, self.output_dim))

        Z = np.einsum('ij,jk->ik', X, self.W) + self.b
        
        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dW = np.einsum('ij,ik->jk', self.X, dZ) / self.X.shape[1] # check here
        db = np.einsum('ij->j', dZ) / self.X.shape[0] # check here
        dX = np.einsum('ij,jk->ik', dZ, self.W.T)

        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db

        return dX

<h2>FLATENNING LAYER</h2>

In [146]:
class Flatenning_Layer:
    def __init__(self):
        self.layer_type = 'Flatten'
    
    def __str__(self):
        return f"{self.layer_type} Layer"
    
    def forward(self, X):
        self.input_shape = X.shape
        return X.reshape((X.shape[0], -1)) # check here
    
    def backward(self, dZ, learning_rate=0.0001):
        dX = np.copy(dZ)
        return dX.reshape(self.input_shape) # check here

<h1>MAX POOLING</h1>

In [147]:
class Max_Pooling:
    def __init__(self, filter_dim, stride):
        self.layer_type = 'Max Pooling'
        self.filter_dim = filter_dim
        self.stride = stride
    
    def __str__(self):
        return f"{self.layer_type} (filter_dim={self.filter_dim}, stride={self.stride})"
    
    def forward(self, X):
        self.X = X

        self.output_dim = (X.shape[1] - self.filter_dim) // self.stride + 1
        
        Z = np.zeros((X.shape[0], self.output_dim, self.output_dim, X.shape[3]))

        for i in range(self.output_dim):
            for j in range(self.output_dim):
                Z[:, i, j, :] = np.max(X[:, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :], axis=(1, 2))

        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dX = np.zeros(self.X.shape)

        for i in range(self.output_dim):
            for j in range(self.output_dim):
                for k in range(dZ.shape[0]):
                    max_index = np.argmax(self.X[k, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :])
                    max_index = np.unravel_index(max_index, (self.filter_dim, self.filter_dim))
                    dX[k, i*self.stride+max_index[0], j*self.stride+max_index[1], :] = dZ[k, i, j, :]
        
        return dX
    

<h1>CONVOLUTION</h1>

In [148]:
class Convolution:
    def __init__(self, num_output_channels, filter_dim, stride=1, padding=0):
        self.layer_type = 'Convolution'
        self.num_output_channels = num_output_channels
        self.filter_dim = filter_dim
        self.stride = stride
        self.padding = padding
        self.W = None
        self.b = None
    
    def __str__(self):
        return f"{self.layer_type} (num_output_channels={self.num_output_channels}, filter_dim={self.filter_dim}, stride={self.stride}, padding={self.padding})"
    
    def forward(self, X):
        self.X = X

        self.output_dim = (X.shape[1] - self.filter_dim + 2 * self.padding) // self.stride + 1

        Z = np.zeros((X.shape[0], self.output_dim, self.output_dim, self.num_output_channels))

        for i in range(self.output_dim):
            for j in range(self.output_dim):
                Z[:, i, j, :] = np.einsum('ij,jkl->kl', X[:, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :], self.W) + self.b
        
        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dW = np.zeros((self.filter_dim, self.filter_dim, self.X.shape[3], self.num_output_channels))
        db = np.zeros((1, 1, 1, self.num_output_channels))
        dX = np.zeros(self.X.shape)

        for i in range(self.output_dim):
            for j in range(self.output_dim):
                dW = dW + np.einsum('ij,ikl->jkl', self.X[:, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :], dZ[:, i, j, :])
                db = db + np.sum(dZ[:, i, j, :], axis=0)
                dX[:, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :] = dX[:, i*self.stride:i*self.stride+self.filter_dim, j*self.stride:j*self.stride+self.filter_dim, :] + np.einsum('ij,jkl->ikl', dZ[:, i, j, :], self.W)
        
        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db

        return dX

<h1>MODEL</h1>

In [149]:
class Model:
    def __init__(self, filePath):
        self.layers = []
        self.filePath = filePath
        self.build_model()

    def __str__(self):
        string = 'MODEL DETAILS:\n\n'
        for i, layer in enumerate(self.layers):
            string += f"Layer {i+1}: {layer}\n"
        return string
    
    def build_model(self):
        #check if file exists
        if not os.path.exists(self.filePath):
            print('File does not exist')
            return
        with open(self.filePath, 'r') as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith('#'):
                    continue

                line = line.strip()
                
                if line == '':
                    continue

                line_split = line.split(' ')
                layer_name = str(line_split[0]).upper()
                
                if layer_name == 'FC':
                    output_dim = int(line_split[1])
                    self.layers.append(Fully_Connected_Layer(output_dim))

                elif layer_name == 'CONV':
                    num_output_channels = int(line_split[1])
                    filter_dim = int(line_split[2])
                    stride = int(line_split[3])
                    padding = int(line_split[4])
                    self.layers.append(Convolution(num_output_channels, filter_dim, stride, padding))

                elif layer_name == 'MAXPOOL':
                    filter_dim = int(line_split[1])
                    stride = int(line_split[2])
                    self.layers.append(Max_Pooling(filter_dim, stride))

                elif layer_name == 'FLATTEN':
                    self.layers.append(Flatenning_Layer())

                elif layer_name == 'RELU':
                    self.layers.append(ReLU_Activation())

                elif layer_name == 'SOFTMAX':
                    self.layers.append(Softmax_Layer())
                
                else:
                    print('Invalid layer name')
                    return
                

    

<h1>BUILD MODEL FROM FILE</h1>

In [150]:
model = Model(MODEL_INIT_FILE)
print(model)

MODEL DETAILS:

Layer 1: Convolution (num_output_channels=5, filter_dim=3, stride=1, padding=0)
Layer 2: Max Pooling (filter_dim=2, stride=2)
Layer 3: ReLU Activation
Layer 4: Flatten Layer
Layer 5: Fully Connected Layer(output_dim=10)
Layer 6: Softmax Layer

